## Upload dos dados para o BigQuery

### Cloud Function

Dataframe de valores reais

In [ ]:
# REQUIREMENTS
# functions-framework==3.*
# google-cloud-bigquery==3.11.2
# pandas==2.0.2
# pyarrow==12.0.1
# gcsfs==2023.6.0

In [ ]:
import functions_framework
from google.cloud import bigquery
import pandas as pd
import gcsfs

# Triggered from a message on a Cloud Pub/Sub topic.
@functions_framework.cloud_event
def upload_real_sao_carlos_weather(cloud_event=None):

    bucket_name = cloud_event.data['message']['attributes']['bucketId']
    obj = cloud_event.data['message']['attributes']['objectId']
    uri = 'gs://' + bucket_name + '/' + obj

    # If for some reason function is triggered with another file abort
    if 'silver/real/real-sao-carlos' not in obj:
        print('Not the expected file')
        return {'Message': 'Not the expected file'}
    
    table_id = 'previsao-tempo-402114.dados_clima.tempo_real'
    df = pd.read_csv(uri)

    client = bigquery.Client()

    job_config = bigquery.LoadJobConfig(
        schema=[
          bigquery.SchemaField('temperature', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('description', bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField('humidity', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('cloudiness', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('rain', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('wind_speedy', bigquery.enums.SqlTypeNames.FLOAT, description='km/h'),
          bigquery.SchemaField('datetime', bigquery.enums.SqlTypeNames.DATETIME)
        ],
        write_disposition='WRITE_APPEND',
        autodetect=False,
        source_format=bigquery.SourceFormat.CSV
    )

    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

    return None


In [ ]:
# gcloud storage buckets notifications create gs://dados-api --topic=upload-real-data-to-bq --event-types=OBJECT_FINALIZE --object-prefix=silver/real/real-sao-carlos

Dataframe de valores previstos

In [ ]:
# REQUIREMENTS
# functions-framework==3.*
# google-cloud-bigquery==3.11.2
# pandas==2.0.2
# pyarrow==12.0.1
# gcsfs==2023.6.0

In [ ]:
import functions_framework
from google.cloud import bigquery
import pandas as pd
import gcsfs

# Triggered from a message on a Cloud Pub/Sub topic.
@functions_framework.cloud_event
def upload_forecast_sao_carlos_weather(cloud_event=None):

    bucket_name = cloud_event.data['message']['attributes']['bucketId']
    obj = cloud_event.data['message']['attributes']['objectId']
    uri = 'gs://' + bucket_name + '/' + obj

    # If for some reason function is triggered with another file abort
    if 'silver/forecast/forecast-sao-carlos' not in obj:
        print('Not the expected file')
        return {'Message': 'Not the expected file'}
    
    table_id = 'previsao-tempo-402114.dados_clima.tempo_forecast'
    df = pd.read_csv(uri)

    client = bigquery.Client()

    job_config = bigquery.LoadJobConfig(
        schema=[
          bigquery.SchemaField('real_date', bigquery.enums.SqlTypeNames.DATE),
          bigquery.SchemaField('date', bigquery.enums.SqlTypeNames.DATE),
          bigquery.SchemaField('max_temperature', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('min_temperature', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('cloudiness', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('rain', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('rain_probability', bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField('wind_speedy', bigquery.enums.SqlTypeNames.FLOAT, description='km/h'),
          bigquery.SchemaField('description', bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField('condition', bigquery.enums.SqlTypeNames.STRING)
        ],
        write_disposition='WRITE_APPEND',
        autodetect=False,
        source_format=bigquery.SourceFormat.CSV
    )

    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

    return None

In [ ]:
# gcloud storage buckets notifications create gs://dados-api --topic=upload-forecast-data-to-bq --event-types=OBJECT_FINALIZE --object-prefix=silver/forecast/forecast-sao-carlos